# 라이브러리 로드

In [1]:
import joblib
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
from tqdm import tqdm

from sklearn.model_selection import cross_val_score
import surprise
from surprise import SVD, Dataset, accuracy, Reader
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split
from surprise.dataset import DatasetAutoFolds

## db 테이블 로드 함수

In [2]:
def find_df(sql):
    # Define connection parameters
    conn = psycopg2.connect(
        host="postgres_db",  # Container name as defined in docker-compose.yml
        database="mydatabase",
        user="myuser",
        password="mypassword"
    )
    # Query data from one of the tables
    # df = pd.read_sql("SELECT * FROM  tn_visit_area_info LIMIT 5;", conn)
    df = pd.read_sql(sql, conn)
    # Close the connection
    df.columns = [x.upper() for x in df.columns]
    conn.close()
    return df
sql = "SELECT * FROM  tn_visit_area_info LIMIT 100;"
df = find_df(sql)

/tmp/ipykernel_105/1441590044.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


## code 데이터 로드

In [3]:
sql ="SELECT * FROM tc_codea"
codea_df = find_df(sql)

sql ="SELECT * FROM tc_codeb"
codeb_df = find_df(sql)

/tmp/ipykernel_105/1441590044.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


# AIHUB 코드 분석

## 1차 전처리

In [4]:
# Get the current timestamp
timestamp = datetime.now()
# visit_E = pd.read_csv('../1.inputdata/tn_visit_area_info_E.csv')
sql = "SELECT * FROM  tn_visit_area_info;"
visit_E = find_df(sql)

id_list=['E']

visit_data_list=[visit_E]

id, visit_area_info = 'E', visit_E
# 관광지 선택
visit_info = visit_area_info[ (visit_area_info['VISIT_AREA_TYPE_CD'] == '1') |
(visit_area_info['VISIT_AREA_TYPE_CD'] == '2') |(visit_area_info['VISIT_AREA_TYPE_CD'] == '3') | (visit_area_info['VISIT_AREA_TYPE_CD'] == '4') |
(visit_area_info['VISIT_AREA_TYPE_CD'] == '5') | (visit_area_info['VISIT_AREA_TYPE_CD'] == '6') |(visit_area_info['VISIT_AREA_TYPE_CD'] == '7') |
 (visit_area_info['VISIT_AREA_TYPE_CD'] == '8')]
visit_info = visit_info.groupby('VISIT_AREA_NM').filter(lambda x: len(x) > 1)
visit_info=visit_info.reset_index(drop = True)

visit_final_E=visit_info

visit_final_E['ratings'] = visit_final_E[['DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION']].astype(float).mean(axis=1)

visit_final_E['TRAVELER_ID'] = visit_final_E['TRAVEL_ID'].str.split('_').str[1]


/tmp/ipykernel_105/1441590044.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


- VISIT_AREA_TYPE_CD 기준으로 필터링(1~8)
  > 테이블 내역기준 관광지만 대상으로
- 지역명 존재여부로 필터링
- ratings 생성
  > 만족도(DGSTFN), 재방문의향(REVISIT_INTENTION), 추천의향(RCMDTN_INTENTION) 평균으로 계산  
- TRAVELER_ID 생성

In [47]:
codea_df.query('CD_A in ["VIS", "DGS", "REP", "REC"]')

,IDX,CD_A,CD_NM,CD_MEMO,CD_MEMO2,DEL_FLAG,ORDER_NUM,PERM_WRITE,PERM_EDIT,PERM_DELETE,INS_DT,EDIT_DT
1,6,VIS,방문지유형코드,,,N,999,N,N,N,2022-07-05 10:45:53,None
9,14,DGS,전반적 만족도 점수,,,N,999,N,N,N,2022-07-05 11:55:48,None
10,15,REP,재방문 의향 점수,,,N,999,N,N,N,2022-07-05 11:56:55,None
11,16,REC,추천의향 점수,,,N,999,N,N,N,2022-07-05 11:58:20,None


In [49]:
codeb_df.query('CD_A in ["VIS"]')

,IDX,CD_A,CD_B,CD_NM,CD_MEMO,CD_MEMO2,DEL_FLAG,ORDER_NUM,INS_DT,EDIT_DT
221,1088,VIS,21,집,,,N,999,2022-07-05 15:52:47,NaT
222,1089,VIS,22,친구/친지집,,,N,999,2022-07-05 15:52:56,2022-07-26 16:05:56
223,1090,VIS,23,사무실,,,N,999,2022-07-05 15:53:05,NaT
224,1091,VIS,24,숙소,,,N,999,2022-07-05 15:53:12,NaT
225,1095,VIS,13,체험 활동 관광지,,,N,15,2022-07-14 15:14:57,NaT
226,1002,VIS,1,자연관광지,,,N,10,2022-07-05 10:48:17,NaT
227,1003,VIS,2,"역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등)",,,N,20,2022-07-05 10:48:24,NaT
228,1005,VIS,3,"문화 시설(공연장, 영화관, 전시관 등)",,,N,30,2022-07-05 11:23:34,NaT
229,1006,VIS,4,"상업지구(거리, 시장, 쇼핑시설)",,,N,40,2022-07-05 11:23:39,NaT
230,1007,VIS,5,"레저/스포츠 관련 시설(스키, 카트, 수상레저)",,,N,50,2022-07-05 11:23:47,NaT


In [50]:
codeb_df.query('CD_A in ["DGS", "REP", "REC"]')

,IDX,CD_A,CD_B,CD_NM,CD_MEMO,CD_MEMO2,DEL_FLAG,ORDER_NUM,INS_DT,EDIT_DT
21,1069,DGS,1,매우 불만족,,,N,999,2022-07-05 11:55:56,NaT
22,1070,DGS,2,불만족,,,N,999,2022-07-05 11:56:01,NaT
23,1071,DGS,3,보통,,,N,999,2022-07-05 11:56:07,NaT
24,1072,DGS,4,만족,,,N,999,2022-07-05 11:56:11,NaT
25,1073,DGS,5,매우만족,,,N,999,2022-07-05 11:56:16,NaT
138,1079,REC,1,전혀 그렇지 않다,,,N,999,2022-07-05 11:58:27,NaT
139,1080,REC,2,그렇지 않다,,,N,999,2022-07-05 11:58:32,NaT
140,1081,REC,3,보통,,,N,999,2022-07-05 11:58:38,NaT
141,1082,REC,4,그렇다,,,N,999,2022-07-05 11:59:30,NaT
142,1083,REC,5,매우 그렇다,,,N,999,2022-07-05 11:59:36,NaT


In [5]:
"""###세부 전처리

####A권역
"""

visit_final_E['SIDO'] = visit_final_E['LOTNO_ADDR'].str.split().str[0]

dfe=visit_final_E
# Group by 'FIRST_WORD' and find the most frequent 'VISIT_AREA_NM' for each group
most_frequent_visits = dfe.groupby('LOTNO_ADDR')['VISIT_AREA_NM'].agg(lambda x: x.mode().iloc[0]).reset_index()

# Merge the most frequent values back to the original DataFrame based on 'FIRST_WORD'
dfe = dfe.merge(most_frequent_visits, on='LOTNO_ADDR', how='left', suffixes=('', '_most_frequent'))

# Update 'VISIT_AREA_NM' with the most frequent values
dfe['VISIT_AREA_NM'] = dfe['VISIT_AREA_NM_most_frequent'].fillna(dfe['VISIT_AREA_NM'])

# Drop temporary columns used for grouping and merging
dfe.drop(columns=['VISIT_AREA_NM_most_frequent'], inplace=True)

dfe[['TRAVELER_ID', 'VISIT_AREA_NM','ratings','SIDO']]

"""####후처리"""

df1 = dfe.rename(columns={'TRAVELER_ID': 'userID','VISIT_AREA_NM': 'itemID','ratings': 'rating'})

df1=df1[['userID','itemID','rating','SIDO']]
# df1.to_csv("../2.preprocessed/dfE.csv")

- 시도명 생성
- 방문지명 최빈값으로 치환
- 유저, 아이템, rating, 시도 만 선택

## 모델

### 그리드서치

In [ ]:
timestamp = datetime.now()

# Print the timestamp
print("Current Timestamp:", timestamp)

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df1[['userID', 'itemID', 'rating']], reader)

algo = SVD()
param_grid = {'n_factors':[50,100,200],'n_epochs': [10,50], 'lr_all': [0.01,0.1], 'reg_all':[0.01,0.1], 'reg_bu':[0.01,0.1],'reg_bi':[0.01,0.1]}
grid = GridSearchCV(SVD, param_grid, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, joblib_verbose= 10)
grid.fit(data=data)

print(grid.best_score['rmse'])
print(grid.best_params['rmse'])


- 그리드서치로 파라미터 튜닝

### 학습 및 모델 저장

In [70]:

# 예상 점수 범위 설정
reader = Reader(line_format='user item rating timestamp', sep='\t')
reader = Reader(rating_scale= (1,5))

# 데이터 불러오기
data = Dataset.load_from_df(df1[['userID', 'itemID', 'rating']], reader)

# 훈련데이터, 테스트 데이터로 나누기
trainset, testset = train_test_split(data, test_size= 0.2, random_state = 42)

testset_df = pd.DataFrame(testset, columns=['userID', 'itemID', 'rating'])

# import pickle
# algo = SVD(n_factors=50, lr_all=0.01, reg_all=0.1, n_epochs=50, reg_bu=0.1, reg_bi=0.1)
# algo.fit(trainset)
# cross_validate(algo=algo, data=data, measures=['RMSE', 'MAE'], cv=10, verbose=True, n_jobs=-1)

# # Specify the file path including the directory where you want to save the model
# file_path = '../4.SaveModel/model/svd_model_E.pkl'
# # Save the model to the specified .pkl file
# with open(file_path, 'wb') as file:
#     pickle.dump(algo, file)

### 모델 검증

In [86]:
loaded_model1 = joblib.load('./src/svd_model_E.pkl')
testset = df1.drop(columns = 'SIDO').to_numpy()
predictions=loaded_model1.test(testset)

prediction_data = []

for uid, iid, true_r, est, _ in predictions:
    prediction_data.append({'userID': uid, 'itemID': iid, 'true_rating': true_r, 'predicted_rating': est})

print(prediction_data)


# Convert the list of dictionaries into a DataFrame
predictions_df_E = pd.DataFrame(prediction_data)

itemID_to_SIDO = df1.set_index('itemID')['SIDO'].to_dict()
predictions_df_E['SIDO'] = predictions_df_E['itemID'].map(itemID_to_SIDO)

predictions_df_E['true_rec']=np.nan
predictions_df_E['est_rec']=np.nan

mean_true_rating = predictions_df_E['true_rating'].mean()
mean_predicted_rating = predictions_df_E['predicted_rating'].mean()
# Set 'true_rec' column based on the condition
predictions_df_E['true_rec'] = np.where(predictions_df_E['true_rating'] > mean_true_rating, 1, 0)

# Set 'est_rec' column based on the condition
predictions_df_E['est_rec'] = np.where(predictions_df_E['predicted_rating'] > mean_predicted_rating, 1, 0)

grouped = predictions_df_E.groupby(['userID', 'true_rec'])

# Iterate through groups
for (user_id, true_rec), group in grouped:
    # Check if 'true_rec' is 1
    if true_rec == 1:
        # Count the occurrences of each 'SIDO' value
        sido_counts = group['SIDO'].value_counts()

        # Get the majority 'SIDO' value
        majority_sido = sido_counts.index[0] if len(sido_counts) > 0 else None

        # Set 'true_rec' to 1 for the majority 'SIDO' and 0 for others
        predictions_df_E.loc[group.index, 'true_rec'] = (group['SIDO'] == majority_sido).astype(int)

grouped = predictions_df_E.groupby(['userID', 'predicted_rating'])

# Iterate through groups
for (user_id, pred_rec), group in grouped:
    if pred_rec == 1:
        # Count the occurrences of each 'SIDO' value
        sido_counts = group['SIDO'].value_counts()

        # Get the majority 'SIDO' value
        majority_sido = sido_counts.index[0] if len(sido_counts) > 0 else None

        # Set 'true_rec' to 1 for the majority 'SIDO' and 0 for others
        predictions_df_E.loc[group.index, 'pred_rec'] = (group['SIDO'] == majority_sido).astype(int)

print(predictions_df_E)
def recall5_calculator(df):
  df_sorted = df.sort_values(by=['userID', 'true_rec', 'predicted_rating'], ascending=[True, False, False])
  # Function to calculate Recall@K for a given user
  def calculate_recall_at_k(user_data, k=5):
      # Count the number of true positives (interest) and total relevant items
      true_positives = user_data['true_rec'].sum()
      total_interest_items = user_data['true_rec'].sum()

      # Count the number of recommended items
      recommended_items = user_data['est_rec'].head(k).sum()

      # Calculate Recall@K
      recall_at_k = true_positives / total_interest_items if total_interest_items > 0 else 0

      return recall_at_k
  recall_at_5_values = df_sorted.groupby('userID').apply(lambda x: calculate_recall_at_k(x, k=5))

  # Calculate average Recall@5 across all users
  average_recall_at_5 = recall_at_5_values.mean()

  return average_recall_at_5

# Print average Recall@5
a=recall5_calculator(predictions_df_E)
print('수도권 권역의 recall@5:'+str(a))

filtered_predictions_df_E = predictions_df_E[predictions_df_E['est_rec'] == 1]

# Group by 'userID' and select top 5 items based on 'predicted_rating'
top_5_items_per_user = (
    filtered_predictions_df_E.groupby('userID')
    .apply(lambda x: x.nlargest(5, 'predicted_rating'))
    .reset_index(drop=True)
)

# Create the recommendation column as a list of itemIDs for each userID
recommendation_lists = top_5_items_per_user.groupby('userID')['itemID'].apply(list).reset_index()

# Rename the columns as userID and recommendation
recommendation_lists.columns = ['userID', 'recommendation']

# Ensure that each list contains at most 5 itemIDs
recommendation_lists['recommendation'] = recommendation_lists['recommendation'].apply(lambda x: x[:5])

# Create the final DataFrame df_E_final
df_E_final = recommendation_lists.copy()
# df_E_final.to_csv('../4.SaveModel/result/testset_output/E_test_ouput.csv')



[{'userID': 'e000004', 'itemID': '화성 관광열차 안내소 연무대 매표소', 'true_rating': 3.6666666666666665, 'predicted_rating': 3.856703481597961}, {'userID': 'e000004', 'itemID': '창룡문', 'true_rating': 4.0, 'predicted_rating': 3.918325682012929}, {'userID': 'e000004', 'itemID': '수원 화성 화홍문', 'true_rating': 3.3333333333333335, 'predicted_rating': 3.856703481597961}, {'userID': 'e000009', 'itemID': '농협안성팜랜드', 'true_rating': 4.0, 'predicted_rating': 4.626469455986174}, {'userID': 'e000009', 'itemID': '스타필드 안성', 'true_rating': 5.0, 'predicted_rating': 4.901026883316274}, {'userID': 'e000009', 'itemID': '평택시 농업 생태원', 'true_rating': 5.0, 'predicted_rating': 4.558471125473589}, {'userID': 'e000010', 'itemID': '롯데월드', 'true_rating': 5.0, 'predicted_rating': 4.803764729922595}, {'userID': 'e000010', 'itemID': '롯데월드', 'true_rating': 5.0, 'predicted_rating': 4.803764729922595}, {'userID': 'e000011', 'itemID': '인사동 문화의 거리', 'true_rating': 5.0, 'predicted_rating': 4.442191389250172}, {'userID': 'e000011', 'itemID': 

In [97]:
user_item_matrix

itemID,21아울렛 구로점,63스퀘어,AK 기흥점,AK& 기흥,AK플라자 광명,AK플라자 금정,AK플라자 분당,AK플라자 수원,AK플라자 홍대,ALT.1 더 현대 서울 현대백화점,...,홍유릉,홍익대학교 대학로 아트센터,화담숲,화성 관광열차 안내소 연무대 매표소,화성 융릉과 건릉,화성행궁,화악산계곡,효원공원 월화원,훈훈 호떡,흥인지문공원
userID,,,,,,,,,,,,,,,,,,,,,
e000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,3.666667,0.0,0.0,0.0,0.0,0.0,0.0
e000009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
e000010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
e000011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
e000013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f000038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
f000458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
g000018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.166667,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
df1.query('userID=="g010556"')

,userID,itemID,rating,SIDO
4111,g010556,방아머리 해변,3.333333,경기
4112,g010556,오이도 빨강등대,5.000000,경기


In [6]:
from personal_key import kakao_api_key

In [7]:
import requests

def convert_address_to_coordinates(address):
    """
    입력받은 주소를 WGS84 좌표계 좌표로 변환
    """

    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    header = {'Authorization': 'KakaoAK ' + kakao_api_key}
 
    r = requests.get(url, headers=header)
 
    if r.status_code == 200:
        lng = float(r.json()["documents"][0]["address"]['x'])
        lat = float(r.json()["documents"][0]["address"]['y'])
    else:
        return None
    
    return lat, lng

In [8]:
def elec_location(region,page_num):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
    params = {'query': region,'page': page_num}
    headers = {"Authorization": "KakaoAK " + kakao_api_key}

    places = requests.get(url, params=params, headers=headers).json()['documents']
    # total = requests.get(url, params=params, headers=headers).json()['meta']['total_count']
    # if total > 45:
    #     print(total,'개 중 45개 데이터밖에 가져오지 못했습니다!')
    # else :
    #     print('모든 데이터를 가져왔습니다!')
    return places

In [9]:
import re

In [10]:
def find_coordinates_search(keyword):
    a = elec_location(keyword,1)
    df_temp = pd.DataFrame(a)
    out_df = df_temp[df_temp.road_address_name==df_temp.road_address_name.value_counts().idxmax()].head(1)
    x,y = out_df.filter(regex='^x|^y').values[0]
    sido = out_df.address_name.values[0].split()[0]
    return x, y, sido

In [11]:
out_list = ['삼일로52', '제부리', '필승로 200']
area_coord= visit_final_E.filter(regex = 'VISIT_AREA_NM|COORD|ADDR|SIDO').groupby('VISIT_AREA_NM').first().reset_index().assign(
    nan_coord = lambda df: df.X_COORD.isna(),
    search_keyword = lambda df: df.apply(lambda row: re.search('[가-힣].*', row.VISIT_AREA_NM).group() if (row.ROAD_NM_ADDR == None) or (row.ROAD_NM_ADDR in out_list) else  row.ROAD_NM_ADDR, axis=1)
)

non_coord = area_coord.query('nan_coord').assign(api_values = lambda df: df.search_keyword.apply(lambda x: find_coordinates_search(x)))\
.assign(X_COORD= lambda df: df.api_values.apply(lambda x: x[0]), 
        Y_COORD= lambda df: df.api_values.apply(lambda x: x[1]),
        SIDO= lambda df: df.api_values.apply(lambda x: x[2]))
exist_coord = area_coord.query('not nan_coord').assign(SIDO = lambda df: df.SIDO.apply(lambda x: '인천' if x == None else x))

area_coord_f = pd.concat([non_coord, exist_coord]).reset_index(drop=True).filter(regex='VISIT_AREA_NM|X_COORD|Y_COORD|SIDO')

In [13]:
area_coord_f.to_csv('area_xy_total.csv', index = False, encoding='utf-8-sig')

In [260]:
area_coord_f

,VISIT_AREA_NM,X_COORD,Y_COORD,SIDO
0,AK 기흥점,127.11680289043568,37.27470975958471,경기
1,ALT.1 더 현대 서울 현대백화점,126.92844612411159,37.525872071029134,서울
2,가납리 비행장,127.020285015859,37.779680819579,경기
3,가로수길,127.022764118441,37.5196385944893,서울
4,계곡,127.36375565272148,37.74945605823188,경기
...,...,...,...,...
790,화악산계곡,127.5241749,37.9701249,경기
791,효원공원,127.0382717,37.264256,경기
792,효원공원 월화원,127.0377333,37.2652491,경기
793,훈훈 호떡,126.9063461,37.5559896,서울


In [258]:
pd.concat([non_coord, exist_coord]).reset_index(drop=True).filter(regex='VISIT_AREA_NM|X_COORD|Y_COORD|SIDO')

,VISIT_AREA_NM,X_COORD,Y_COORD,SIDO
553,월미문화로,126.596334,37.47145,None


In [249]:
out_df.address_name.values[0]

'경기 용인시 기흥구 구갈동 234-3'

In [263]:
df1.drop(columns = 'SIDO').merge(area_coord_f, left_on='itemID', right_on='VISIT_AREA_NM')


,userID,itemID,rating,VISIT_AREA_NM,X_COORD,Y_COORD,SIDO
0,e000004,화성 관광열차 안내소 연무대 매표소,3.666667,화성 관광열차 안내소 연무대 매표소,127.0233392,37.2878779,경기
1,e000004,창룡문,4.000000,창룡문,127.0251435,37.287791,경기
2,e000004,수원 화성 화홍문,3.333333,수원 화성 화홍문,127.0176261,37.2875455,경기
3,e000009,농협안성팜랜드,4.000000,농협안성팜랜드,127.1935166,36.9913166,경기
4,e000009,스타필드 안성,5.000000,스타필드 안성,127.1470947,36.9950254,경기
...,...,...,...,...,...,...,...
4108,g000018,AK플라자 수원,3.333333,AK플라자 수원,127.00069,37.2655524,경기
4109,g000018,AK플라자 수원,3.000000,AK플라자 수원,127.00069,37.2655524,경기
4110,g005600,국립과천과학관,5.000000,국립과천과학관,127.0058182,37.4380317,경기
4111,g010556,방아머리 해변,3.333333,방아머리 해변,126.5767674,37.2891527,경기


In [265]:
df1.drop(columns = 'SIDO').to_csv('preprocessed.csv', index=False, encoding='utf-8-sig')

In [ ]:
input_sample = 

In [104]:
dfc_pivot

item_id,21아울렛 구로점,63스퀘어,AK 기흥점,AK& 기흥,AK플라자 광명,AK플라자 금정,AK플라자 분당,AK플라자 수원,AK플라자 홍대,ALT.1 더 현대 서울 현대백화점,...,홍유릉,홍익대학교 대학로 아트센터,화담숲,화성 관광열차 안내소 연무대 매표소,화성 융릉과 건릉,화성행궁,화악산계곡,효원공원 월화원,훈훈 호떡,흥인지문공원
user_id,,,,,,,,,,,,,,,,,,,,,
e000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,3.856703,0.0,0.0,0.0,0.0,0.0,0.0
e000009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
e000010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
e000011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
e000013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f000038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
f000458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
g000018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.452733,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:

dataset = df1.drop('SIDO',axis=1).values.tolist()
sido_df=df1[['itemID', 'SIDO']].drop_duplicates()
predictions = loaded_model1.test(dataset)

# 예측 결과를 데이터프레임으로 변환
result_dfc = pd.DataFrame(predictions, columns=['user_id', 'item_id', 'actual_rating', 'predicted_rating', 'details'])

# 필요한 컬럼만 선택 (user_id, item_id, actual_rating, predicted_rating)
result_dfc = result_dfc[['user_id', 'item_id', 'actual_rating', 'predicted_rating']]

dfc_pivot=result_dfc.pivot_table('predicted_rating', index='user_id', columns='item_id').fillna(0)

user_item_matrix = df1.pivot_table(index = 'userID', columns = 'itemID', values = 'rating').fillna(0)

# 이미 방문한 숙박 업소 제외하고 추천
recommendations = []
for idx, user in enumerate(user_item_matrix.index):
    # 해당 사용자가 방문한 관광지
    applied_accs = set(user_item_matrix.loc[user][user_item_matrix.loc[user] != 0].index)

    sorted_acc_indices = dfc_pivot.iloc[idx].argsort()[::-1]
    recommended_accs = [acc for acc in user_item_matrix.columns[sorted_acc_indices] if acc not in applied_accs][:5]
    for acc in recommended_accs:
        recommendations.append([user, acc])

top_recommendations = pd.DataFrame(recommendations, columns=['userID', 'itemID'])
top_recommendations_with_sido = pd.merge(top_recommendations, sido_df, on='itemID', how='left')

def filter_majority_sido(group):
    sido_counts = group['SIDO'].value_counts()
    if len(sido_counts) > 0:
        majority_sido = sido_counts.idxmax()
        group = group[group['SIDO'] == majority_sido]
    return group

# Group top_recommendations_with_sido by 'userID' and apply the filter_majority_sido function
filtered_top_recommendations = top_recommendations_with_sido.groupby('userID').apply(filter_majority_sido)

# Reset index after applying the function
filtered_top_recommendations.reset_index(drop=True, inplace=True)

# filtered_top_recommendations.to_csv('../4.SaveModel/result/final_output/E_top_recommendations.csv')

# Get the current timestamp
timestamp = datetime.now()

# Print the timestamp
print("Current Timestamp:", timestamp)

Current Timestamp: 2024-10-29 13:50:59.057151


/tmp/ipykernel_21/1579316340.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_top_recommendations = top_recommendations_with_sido.groupby('userID').apply(filter_majority_sido)


In [89]:
filtered_top_recommendations

,userID,itemID,SIDO
0,e000004,방화수류정,경기
1,e000004,방아머리 해변,경기
2,e000004,바다 향기수목원,경기
3,e000009,배곧 생명공원,경기
4,e000009,방화수류정,경기
...,...,...,...
6438,g005600,방화수류정,경기
6439,g005600,방아머리 해변,경기
6440,g010556,배곧 생명공원,경기
6441,g010556,방화수류정,경기


In [108]:
visit_final_E

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,ratings,TRAVELER_ID,SIDO
0,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30,경기 수원시 팔달구 창룡대로103번길 20,경기 수원시 팔달구 매향동 3-32,127.0233392,37.2878779,...,N,10,None,4,3,4,None,3.666667,e000004,경기
1,2304300003,e_e000004,3,창룡문,2023-04-30,2023-04-30,None,경기 수원시 팔달구 남수동,127.0251435,37.287791,...,N,1,None,4,4,4,None,4.000000,e000004,경기
2,2304300004,e_e000004,4,수원 화성 화홍문,2023-04-30,2023-04-30,None,경기 수원시 팔달구 북수동 9000-1,127.0176261,37.2875455,...,N,10,None,4,3,3,None,3.333333,e000004,경기
3,2304290002,e_e000009,2,농협안성팜랜드,2023-04-29,2023-04-29,경기 안성시 공도읍 대신두길 28,경기 안성시 공도읍 신두리 451,127.1935166,36.9913166,...,N,1,None,4,4,4,None,4.000000,e000009,경기
4,2304290003,e_e000009,3,스타필드 안성,2023-04-29,2023-04-29,경기 안성시 공도읍 서동대로 3930-39,경기 안성시 공도읍 진사리 354,127.1470947,36.9950254,...,N,1,None,5,5,5,None,5.000000,e000009,경기
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4108,2304290006,g_g000018,5,AK플라자 수원,2023-04-29,2023-04-29,경기 수원시 팔달구 덕영대로 924,경기 수원시 팔달구 매산로1가 18,127.00069,37.2655524,...,Y,9,None,3,3,4,None,3.333333,g000018,경기
4109,2304290010,g_g000018,9,AK플라자 수원,2023-04-29,2023-04-29,경기 수원시 팔달구 덕영대로 924,경기 수원시 팔달구 매산로1가 18,127.00069,37.2655524,...,Y,7,None,3,3,3,None,3.000000,g000018,경기
4110,2308230002,g_g005600,2,국립과천과학관,2023-08-23,2023-08-23,경기 과천시 상하벌로 110,경기 과천시 과천동 758,127.0058182,37.4380317,...,Y,10,None,5,5,5,None,5.000000,g005600,경기
4111,2309020005,g_g010556,4,방아머리 해변,2023-09-02,2023-09-02,None,경기 안산시 단원구 대부북동,126.5767674,37.2891527,...,N,1,None,3,4,3,None,3.333333,g010556,경기


In [ ]:
sql = "SELECT FROM tn_poi_master WHERE"
find_df()